In [1]:
import rpy2.robjects as robjects
import numpy as np
import os as os
from rpy2.robjects import numpy2ri
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.special import expit
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
# Cell 1: Import the module
#from gp_softmax_torch_works_play import *
from gp_softmax_torch_works_play_matrprev import *
## first softmax version
%load_ext autoreload
%autoreload 2
# Specify the path to your data files
data_path = "/Users/sarahurbut/tensornoulli_ehr_new/data"

# Activate automatic conversion between R and NumPy arrays
numpy2ri.activate()

# Load data saved as .rds files
Y = np.array(robjects.r['readRDS'](os.path.join(data_path, 'Y.rds')))
E = np.array(robjects.r['readRDS'](os.path.join(data_path, 'event_for_aladynoulli.rds')))
G = np.array(robjects.r['readRDS'](os.path.join(data_path, 'prs.rds')))
# Print shapes to verify
print("Y shape:", Y.shape)
print("E shape:", E.shape)
print("G shape:", G.shape)

E = E.astype(int)

# G should be float64
G = G.astype(float)

G.shape
G = G.T
print("G shape after transposition:", G.shape)

# Print the first few elements of the first row to verify
print("G first elements of first individual after transposition:", G[0, :5])

# Get dimensions
N, D, T = Y_tensor.shape
P = G_tensor.shape[1]
T = int(E_tensor.max() + 1)  # 0-indexed time
K = 10  # number of topics

# Print shapes to verify
print("Tensor shapes:")
print(f"Y: {Y_tensor.shape}")  # [N, D]
print(f"E: {E_tensor.shape}")  # [N, D]
print(f"G: {G_tensor.shape}")  # [N, P]
print(f"Time range: 0 to {T-1}")
print(T)


Y shape: (9661, 348, 51)
E shape: (9661, 348)
G shape: (36, 9661)
G shape after transposition: (9661, 36)
G first elements of first individual after transposition: [-0.09865071 -0.61293518 -0.71439616 -0.49112834  0.4647725 ]


NameError: name 'Y_tensor' is not defined